Author: Wynona Lam  
Role: SULI Intern  
Date: April 2024  

Goal: To calculate CO2 emissions related to the various scenarios described in the paper: "Techno-economic analysis of renewable energy generation at the South Pole".

In [1]:
import south_pole_traverse as sp

Emissions associated with 1 year of diesel fuel used in the South Pole which is 124k gallons of disel per the Report
Scope: Embodied emissions of the fuel needed at the South Pole and used within the transport of the fuel + emissions released during the transportation of said fuel (combustion)

In [2]:
#124k gallons of diesel weights approx 800k lbs aka 400 tons (124k gathered from report, weight calculation from Google)
emissions_first_leg = sp.calculate_tanker_emissions(6900, 400, 1)
emissions_second_leg = sp.calculate_tanker_emissions(2500, 400/2, 2) #This function assumes the cargo weight is carried by all vehicles aka in this case the 2 tankers (not true)
emissions_third_leg = sp.calculate_truck_emissions(1030, 80, 9) #Each vehicle pulls between 160k-200k lbs of fuel per the Report
emissions_fuel_used = sp.calculate_fuelused_emissions(11730, 9270)
emissions_fuel_transported = sp.calculate_emissions_from_diesel(124000) #in gallons

In [8]:
total_emissions = sp.sum_emissions(
    emissions_first_leg,
    emissions_second_leg,
    emissions_third_leg,
    emissions_fuel_used,
    emissions_fuel_transported  # Assuming emissions_fuel_transported has been calculated and is a flat dict
)

print(total_emissions)
co2_emissions = sp.calculate_co2_equivalent(total_emissions)
print("Grams of CO2 released: " + str(co2_emissions))


{'CO2': 233759086.75851363, 'CH4': 1922227.9395642178, 'N2O': 4325.249834421531}
Grams of CO2 released: 292222272.5623244


Final Metrics: 292222272g of CO2 or 292222.27kg are released during 1 year of diesel fuel transported to the south pole. This does not include the emissions released when the fuel is actually used at the base. Now let's roughly calculate the CO2 released during the transportation and manufacturing of the renewable energy sources in various scenarios.  

Scenario A was unrealistic, thus not calculated at all.  

Scenario B: PV System Size (Kw-dc) = 0, Wind System Size (KW) = 780,  BESS power (kw) = 200,  BESS energy (kwh) = 3310,  Diesel fuel (gallons) = 9500  

Scenario C: PV System Size (Kw-dc) = 180  Wind System Size (KW) = 570  BESS power (kw) = 180  BESS energy (kwh) = 3410  Diesel fuel (gallons) = 5600  
  
Scenario D: PV System Size (Kw-dc) = 120  Wind System Size (KW) = 600  BESS power (kw) = 180  BESS energy (kwh) = 12570  Diesel fuel (gallons) = 0  
  
Scenario E: PV System Size (Kw-dc) = 200  Wind System Size (KW) = 580  BESS power (kw) = 200  BESS energy (kwh) = 2210  Diesel fuel (gallons) = 8500  

Typically the wind turbines and lithium ion BESS storage are shipped via LC-130. Max cargo load of a LC-130 is 45k lbs. The panels can be sledded.  

In [2]:
#Scenario B
emissions_scenario_b = sp.transportation_scenario_emissions(0, 100, 780, 200, 3310, 9500)
embodied_emissions_b = sp.embodied_renewable_emissions(0, 780, 3310, 9500)
total_emissions_b  = sp.consolidate_scenario_emissions(emissions_scenario_b, embodied_emissions_b)
co2_emissions_b = sp.calculate_co2_equivalent(total_emissions_b)
print("Grams of CO2 released for B: " + str(co2_emissions_b))

#Scenario C
emissions_scenario_c = sp.transportation_scenario_emissions(180, 100, 580, 180, 3410, 5600)
embodied_emissions_c = sp.embodied_renewable_emissions(180, 570, 3410, 5600)
total_emissions_c  = sp.consolidate_scenario_emissions(emissions_scenario_c, embodied_emissions_c)
co2_emissions_c = sp.calculate_co2_equivalent(total_emissions_c)
print("Grams of CO2 released for C: " + str(co2_emissions_c))

#Scenario D
emissions_scenario_d = sp.transportation_scenario_emissions(120, 100, 600, 180, 12570, 0)
embodied_emissions_d = sp.embodied_renewable_emissions(120, 600, 12570, 0)
total_emissions_d  = sp.consolidate_scenario_emissions(emissions_scenario_d, embodied_emissions_d)
co2_emissions_d = sp.calculate_co2_equivalent(total_emissions_d)
print("Grams of CO2 released for D: " + str(co2_emissions_d))

#Scenario E
emissions_scenario_e = sp.transportation_scenario_emissions(200, 100, 580, 200, 2210, 8500)
embodied_emissions_e = sp.embodied_renewable_emissions(200, 580, 2210, 8500)
total_emissions_e  = sp.consolidate_scenario_emissions(emissions_scenario_e, embodied_emissions_e)
co2_emissions_e = sp.calculate_co2_equivalent(total_emissions_e)
print("Grams of CO2 released for E: " + str(co2_emissions_e))

Grams of CO2 released for B: 1329610389.1508195
Grams of CO2 released for C: 1397310612.1105995
Grams of CO2 released for D: 3354551306.940117
Grams of CO2 released for E: 1168613072.375611


The final metrics for each scenario are as such:  

- Scenario B: 1329610389.15g or 1329610.389 kg co2   
- Scenario C: 1397310612.11g or 1397310.612 kg co2    
- Scenario D: 3354551306.94g or 3354551.307 kg co2  
- Sceanrio E: 1168613072.37g or 1168613.072 kg co2  

is released. This is not including the maintenance and other smaller parts of the renewable energy system.

Final Conclusion & Assumptions:  

It would take each scenario...

- Scenario B: Approximately 4.55 years  
- Scenario C: Approximately 4.78 years  
- Scenario D: Approximately 11.48 years  
- Scenario E: Approximately 4.00 years  

to pay off the carbon debt associated with the R.E vs just shipping the diesel fuel (assuming it stays at 124k gallons per year)

Assumptions of the code:
- Icebreakers are modeled as an Ocean Tanker
- All case quadtracs, prinoths, and LC-130s are modeled as heavy, heavy duty trucks 
- Wind turbines are assumed to take 7 LC-130 trips to get from McMurdo to South Pole
- When vehicles pull cargo weighing more than their max load, fuel efficiency and emissions are presumed to be not affected
- The trip to to destination emissions are calculated using the BTU/ton-mile metric in GREET vs when on the way back with an empty load, calculated with the fuel efficiency
- The emissions from the fuel used during the SpoT trip assumes constant specific fuel efficiency and MPG regardless of cargo load

Therefore the max cargo/payload for each vehicle and the associated fuel efficiency are violated through these assumptions; thus, the CO2 emissions are not extremely accurate. The paper and code go into depth regarding how assumptions affect the calculations and where some of the constants used are found.

These are extra calculations for unit measurement:

In [7]:
emissions = sp.calculate_truck_emissions(1030, 0.00110231, 9)
total_co2_for_1kg_spot = sp.calculate_co2_equivalent(emissions) 
print(total_co2_for_1kg_spot)
print(emissions)

11712.982411868461
{'CO2': 11290.705770409522, 'CH4': 13.760618550822628, 'N2O': 0.04472603899056617}


CO2 for the SpoT (sled delivery) portion of the Trip for 1kg, using all 9 vehicles: 11290g of CO2/kg shipped (quite high, but this is the round trip figure + turns the n2o and ch4 to co2?). The CO2 for the C-130 is the same as above since this program assumes it to be a truck as well.

Now, let's calculate the CO2 (g)/kg for the tanker shipment from LA to McMurdo

In [4]:
first_leg_emissions = sp.calculate_tanker_emissions(6900, .00110231, 1)
second_leg_emissions = sp.calculate_tanker_emissions(2415, .000551155, 2) 
first_leg_co2 = sp.calculate_co2_equivalent(first_leg_emissions)
second_leg_co2 = sp.calculate_co2_equivalent(second_leg_emissions)

total_co2_la_to_ant = first_leg_co2+second_leg_co2
total_co2_la_to_ant

11702498.604347453

For the trip from LA to McMurdo via 1 tanker for the first leg and two tankers (icebreaker + tanker) for the second leg carrying 1kg emits 11702498g of CO2 (again quite high, but this is a round trip?)